# [Репозиторий](https://github.com/Lynxtail/Additional-chapters-of-fundamental-training)

## Задание G, вариант 1

Выполнить хэширование с помощью $T$-функции $H : \{0, 1\}^m \times \{0, 1\}^m \rightarrow \{0, 1\}^m$. Для построения функции $H$ нужно использовать корректно определенные операции (арифметические, поразрядные логические, экспоненцирование, деление).

In [45]:
import bitstring as bs
import matplotlib as plt
import random

### Определение операций

In [46]:
# наложение маски a and C
def mask_(a:bs.BitArray, C:bs.BitArray='0b1111') -> bs.BitArray:
    return a & C

In [144]:
# экспоненцирование x |^ y := (1 + 2x)^y
def expo_(x:int, y:int, m:int) -> int:
    return pow(1 + 2 * x, y, 2**m)

In [48]:
# деление x/y := x(y |^ (-1)) = x / (1 + 2y)
def divide_(x:int, y:int, m:int) -> int:
    return x * expo_(y, -1, m)

In [49]:
# редуцирование a mod 2^n
def reduce_(a:int, n:int) -> int:
    return a % 2**n

### Вспомогательные функции

Построение $T-$ функции $H:\{0, 1\}^m \times \{0,1\}^m \rightarrow \{0,1\}^m$

Пример такой функции: $$(x \oplus y) + (1 + x^2)$$

In [345]:
# H - T-функция на {0, 1}^m, где m - длина
def H_1(x:bs.BitArray, y:bs.BitArray, m:int) -> bs.BitArray:
    # e.g.
    ans = bs.BitArray(bin((x ^ y).u + (1 + pow(x.u, 2, 2**m))))
    if len(ans) > m:
        ans.overwrite('0b' + ''.join(['0' for _ in range(len(ans) - m)]), 0)
    ans = bs.BitArray(uint=ans.u, length=m)
    return ans

Собственная функция: $$(x \& y)^{x \oplus y} + (x^4 \cdot y \uparrow x)$$

In [346]:
# H - T-функция на {0, 1}^m, где m - длина
def H_2(x:bs.BitArray, y:bs.BitArray, m:int) -> bs.BitArray:
    ans = bs.BitArray(bin(pow((x & y).u, (x ^ y).u, 2**m) + (pow(x.u, 4, 2**m) * expo_(x.u, y.u, m))))
    if len(ans) > m:
        ans.overwrite('0b' + ''.join(['0' for _ in range(len(ans) - m)]), 0)
    ans = bs.BitArray(uint=ans.u, length=m)
    return ans

Построение графика

In [333]:
# построение графика T-функции
def draw(ans:list[int], x:list[int], y:list[int], title:str, m:int) -> None:
    plt.scatter(x, y, s=1)
    plt.title(f'H(x, y) = {title}, m = {m}')

Разбиение сообщения W на блоки битов длиной $m$

In [336]:
# разбиение сообщения w на блоки a_i длины m каждый
# w = [a_k]...[a_2][a_1]
def split_w(m:int, w:str) -> bs.BitArray:

    # если сообщение не в виде бинарной строки
    if len(set(w).difference({'0', '1'})) != 0:
        a = '.'.join(map(bin, bytearray(w, 'utf8')))[2:].replace('.0b', '')
    # если сообщение в виде бинарной строки
    else: a = w

    # разбиение
    a = [a[max(i - m, 0) : i] for i in range(len(a), 0, -m)]

    # добавление незначащих нулей к крайнему a
    if len(a[-1]) < m:
        a[-1] = ''.join(['0']*(m - len(a[-1]))) + a[-1]

    for i in range(len(a)):
        a[i] = bs.BitArray(bin=a[i])

    print('a = { ', end='')
    [print(f'[{item.b}]', end=' ') for item in a]
    print('}')

    return a

a = { [111] [010] [011] [011] [000] }


Генерация $h_0$ и вычисление значений функции $H(x, y)$:

In [344]:
def calculate_h(w:str, m:int, a:bs.BitArray, fun:str) -> bs.BitArray:
    # h_{i+1} = H(a_{i+1}, h_i), i = 0, 1, ...
    # h_k - хеш-значение сообщения

    # h_0 = [random.randint(0, 1) for _ in range(m)]
    # h_0 = ''.join(map(str, h_0))
    h_0 = '010'
    h_0 = bs.BitArray(bin=h_0)

    k = len(a)
    h = [h_0]
    
    print(f'Исходное сообщение:\n{w}')
    print(f'Используемая функция: {fun}')
    print(f'h0 = {h_0.b}')
    for i in range(k):
        print(f'h{i + 1} = H({a[i].b, h[-1].b})', end=' = ')
        h.append(H_1(a[i], h[-1], m))
        print(h[-1].b)

H(x, y) = (x XOR y) + (1 + x^2)
h0 = 010
h1 = H(('111', '010')) = 111
h2 = H(('010', '111')) = 010
h3 = H(('011', '010')) = 011
h4 = H(('011', '011')) = 010
h5 = H(('000', '010')) = 011


### Основная часть

Ввод исходного сообщения и длины $m$ блоков разбиения:

In [347]:
# w = input("Введите сообщение: ")
# m = int(input("Введите длину m: "))

In [349]:
# тестовый пример
w = '0011011010111'
m = 3
fun = '(x XOR y) + (1 + x^2)'

In [ ]:
a = split_w(m, w)
h = calculate_h(w, m, a, fun)
draw(list(map(bs.BitArray.u, h[1:])), list(map(bs.BitArray.u, a)), list(map(bs.BitArray.u, h)), fun, m)
plt.show()